<img src="GEOS_Logo.png" width="700" />

# Step 1 : <font color=blue>"download_files.ipynb"</font>
#### Aug 15, 2021  <font color=red>(still working)</font> 
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)



> list_full.dat (904 stations) and list_extra.dat (extra 3 stations) \
> total is 907 \
> At some point, a code that downloads (a)a station ID list and (b)a list of their coordinates will be ready \
> And then it will count how many stations, this GPS2FNL algorithm will process and pass this info to the later codes. \
> metadata is needed

> input is : list_default.dat


### **`GPS data processed by the NGL is available at:`**  [click here](http://geodesy.unr.edu)

> On the right side of the website, find <font color=red>"Downloadable Lists of GPS Data Holdings:"</font> \
> This algorithm uses data from "stations with 24 hour sample rate solutions, final orbits, 2 week latency." 



In [1]:
# Ask if a user wants all stations (except MAGNET) or NOTA stations 
# If one choose to use all stations, the vertical position analysis must be done individually. 

In [2]:
# Location Boundary
lon_min=235.43
lon_max=246.99
lat_min=31.31
lat_max=42.99

# Later in this code, the threshold of minimum daily position estimates will be determined. 
# > Minimum number of daily position estimates:
# >> if you want the NOTA stations, threshold_num = 0 (to include all NOTA stations regardless of data)
# >>> Other than that, threshold_num = 200 ()

In [3]:
#import modules
import numpy as np
import pandas as pd
import urllib

1.  Read the station list
> url_lists="http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt"

2. Download metadate (discontinuous steps)
> url_metadata="http://geodesy.unr.edu/NGLStationPages/steps.txt"



In [4]:
# A list of GPS stations 
url_list="http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt" # URL
list_file="list_NGL.txt" # file name

# metadata for steps
url_metadata="http://geodesy.unr.edu/NGLStationPages/steps.txt" # URL
metadata_file="steps_NGL.txt" # file name

In [5]:
# Download the list of GPS stations (final orbits) and metadata 
urllib.request.urlretrieve(url_list, list_file)
urllib.request.urlretrieve(url_metadata, metadata_file)
print("Downloaded a GPS list and metadata for steps")

Downloaded a GPS list and metadata for steps


In [6]:
# Read the station list.
df_original_list=pd.read_csv(list_file ,header=None, sep=r'(?:,|\s+)', 
                comment='#', engine='python',
                names=['Sta','lat','lon','3','4','5','6','7','8','9','NumSol','11','12','13'])


# Here the first 4 columns will be used.
df_four_cols=df_original_list.loc[1:len(df_original_list),['Sta','lat','lon','NumSol']] #first three columns
df_four_cols=df_four_cols.reset_index(drop=True)

# str to float
df_four_cols['lon'] = df_four_cols['lon'].astype(float)
df_four_cols['lat'] = df_four_cols['lat'].astype(float)
df_four_cols['NumSol'] = df_four_cols['NumSol'].astype(int)



In [18]:
# While the user input is not either yes or no, this loop will repeat

while True:      
    print(" > If you want to include all GPS stations in California, type 'yes'.")
    print(" >> If you type 'no', only NOTA stations will be used.")
    YESorNO=input(" type 'yes' or 'no' ")
    YESorNO=YESorNO.lower()
    
    while YESorNO not in ("yes","no"):
        print("*** Please type 'yes' or 'no' ***")
        YESorNO=input(" type 'yes' or 'no' ")
        YESorNO=YESorNO.lower()
        
    if YESorNO == "no" or YESorNO == "yes":
        break 
        #break the while loop
         

# YES! => all stations except MAGNET stations will be included
if YESorNO.startswith('y'): 
    # Select stations within the region of interest.
    # > lon_min,lon_max,lat_min,lat_max must be defined.
    
    threshold_num=200
    df_interest=df_four_cols.loc[(df_four_cols['lat'] >= lat_min)\
                            & (df_four_cols['lat'] <= lat_max)\
                            & (df_four_cols['lon'] >= lon_min)\
                            & (df_four_cols['lon'] <= lon_max)\
                            & (df_four_cols['NumSol'] >= threshold_num)]

    df_interest=df_interest.reset_index(drop=True)
    
    #REMOVE MAGNET STATIONS FROM THE LIST
    URL_MAGNET="http://geodesy.unr.edu/magnet/Table2web.html"
    df_MAGNET=pd.read_html(URL_MAGNET, header=1, flavor = 'bs4')
    df_MAGNET=df_MAGNET[0]
    df_MAGNET.columns=['Sta','1','2','3','4','5','6']
    df_no_MAGNAT = df_interest[~df_interest['Sta'].isin(df_MAGNET['Sta'])].reset_index(drop=True)
    df_final = df_no_MAGNAT
    
# NO!
else: 
    threshold_num=0
    df_interest=df_four_cols.loc[(df_four_cols['lat'] >= lat_min)\
                            & (df_four_cols['lat'] <= lat_max)\
                            & (df_four_cols['lon'] >= lon_min)\
                            & (df_four_cols['lon'] <= lon_max)\
                            & (df_four_cols['NumSol'] >= threshold_num)]

    df_interest=df_interest.reset_index(drop=True)
    list_default="list_default.dat"
    df_default=pd.read_csv(list_default, header=None, names=['Sta'])
    df_NOTA = df_interest[df_interest['Sta'].isin(df_default['Sta'])].reset_index(drop=True)
    df_final = df_NOTA

 > If you want to include all GPS stations in California, type 'yes'.
 >> If you type 'no', only NOTA stations will be used.


 type 'yes' or 'no'  yes


In [23]:
df_station=df_final.loc[:,['Sta']]

In [27]:
df_coor=df_final.loc[:,['lon','lat']]

In [1]:
#SAVE FILES 
# coordinates and station list
df_station.to_csv('station_list_test.dat',header=None, index=None,sep=' ',float_format='%g')
df_coor.to_csv('coordinate_list_test.dat',header=None, index=None, sep=' ',float_format='%g')

NameError: name 'df_station' is not defined